In [29]:
import random

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.parse import urlparse, urlencode
import re
import time
from tqdm import tqdm
from urllib.parse import urlparse
from urllib.parse import parse_qs
import csv

In [30]:
NUM_PAGES = 200
BASE_URL = 'https://www.domain.com.au'

In [31]:
urls = []

In [32]:
user_agents = [
    "PostmanRuntime/7.6.0",
    "PostmanRuntime/6.6.0",
    "PostmanRuntime/7.5.0",
    "PostmanRuntime/7.2.0",
    "PostmanRuntime/5.6.0",
    "PostmanRuntime/4.1.0",
]

In [ ]:
postcodes = list(range(3000, 3200)) + [3800, 3801]

In [33]:
relative_url = BASE_URL + '/rent/?sort=dateupdated-desc&state=vic'
for i in tqdm(range(1, NUM_PAGES + 1)):
    if i % 50 == 0:
        # time.sleep(60)
        pass

    url = relative_url + '&page=' + str(i)
    # print(url)
    req = Request(url, headers={'User-Agent': random.choice(user_agents)})
    html = urlopen(req).read()
    soup = BeautifulSoup(html, "lxml")

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = (
        soup.findAll(
            "a",
            {"class": "address"},
            href=re.compile(f"{BASE_URL}/*"),  # the `*` denotes wildcard any
        )
    )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            urls.append(link['href'])

  3%|▎         | 6/200 [00:03<01:44,  1.86it/s]


KeyboardInterrupt: 

In [ ]:
print(f"Found {len(urls)} urls")

In [35]:
def scrape_property(property_url):
    data = {}

    req = Request(property_url, headers={'User-Agent': random.choice(user_agents)})
    html = urlopen(req).read()

    soup = BeautifulSoup(html, "lxml")

    data["url"] = property_url
    data["price"] = soup.find("div", {"data-testid": "listing-details__summary-title"}).text
    data["address"] = soup.find("h1").text
    data["property_type"] = soup.find("div", {"data-testid": "listing-summary-property-type"}).text

    data["features"] = []
    for feature in soup.find("div", {'data-testid': "property-features-wrapper"}).findAll("span", {
        "data-testid": "property-features-feature"}):
        data["features"].append(feature.text)

    data["summary"] = []
    for s in soup.find("ul", {"data-testid": "listing-summary-strip"}).findAll("li"):
        data["summary"].append(s.text)

    google_maps = soup.find("a", href=re.compile(f"https://www.google.com/maps/dir/*"))['href']
    coords = parse_qs(urlparse(google_maps).query)["destination"][0].split(",")
    data["latitude"] = float(coords[0])
    data["longitude"] = float(coords[1])

    return data

{'url': 'https://www.domain.com.au/8-perth-avenue-albion-vic-3020-17193989',
 'price': '$460',
 'address': '8 Perth Avenue, Albion VIC 3020',
 'features': ['3 Beds', '1 Bath', '− Parking'],
 'property_type': 'House',
 'summary': ['Available from Monday, 16th September 2024', 'Bond $1999'],
 'latitude': -37.7789486,
 'longitude': 144.8150825}